In [1]:
# 1. Core PyTorch first
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118  # (change cu118 if using CPU or other CUDA)

# 2. Utility libs
!pip install matplotlib timm

# 3. Segment Anything
!pip install git+https://github.com/facebookresearch/segment-anything.git

# 4. CLIP
!pip install git+https://github.com/openai/CLIP.git ftfy regex

# 5. OpenCV (with contrib modules for tracker)
!pip install opencv-contrib-python



Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#'


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\hello\appdata\local\temp\pip-req-build-9d_glsd_
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-9d_glsd_'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\hello\appdata\local\temp\pip-req-build-lbtl__lq
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-lbtl__lq'


Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl (45.3 MB)


In [3]:
import os
import cv2
import numpy as np
import torch
import timm
from segment_anything import sam_model_registry, SamPredictor


In [5]:
!curl.exe -L "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth" -o "sam_vit_b.pth"
does the big code

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  357M    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1  357M    1 4122k    0     0  3222k      0  0:01:53  0:00:01  0:01:52 3274k
  1  357M    1 7290k    0     0  3206k      0  0:01:54  0:00:02  0:01:52 3236k
  2  357M    2 10.1M    0     0  2991k      0  0:02:02  0:00:03  0:01:59 3009k
  3  357M    3 12.3M    0     0  2952k      0  0:02:04  0:00:04  0:02:00 2966k
  4  357M    4 15.2M    0     0  2967k      0  0:02:03  0:00:05  0:01:58 3161k
  5  357M    5 18.2M    0     0  2973k      0  0:02:03  0:00:06  0:01:57 2909k
  6  357M    6 22.0M    0     0  3104k      0  0:01:57  0:00:07  0:01:50 3058k
  7  357M    7 25.9M    0     0  3208k      0  0:01:54  0:00:08  0:01:46 3363k
  8  357M    8 28.7M    0     0  3171k      0  0:01

In [7]:
import os
print(os.path.exists("sam_vit_b.pth"))  # Should be True


True


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model (point to the checkpoint you downloaded)
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam.to(device)
predictor = SamPredictor(sam)

In [ ]:
VIDEO_SRC = "C:\\Users\\Hello\\Downloads\\184734-873923034_small.mp4"
# open video
cap = cv2.VideoCapture(VIDEO_SRC)
if not cap.isOpened():
    print("Cannot open video")
    exit()

# get first frame for click
ret, first_frame = cap.read()
if not ret:
    print("Cannot read first frame")
    exit()

click_point = None
def mouse_click(event, x, y, flags, param):
    global click_point
    if event == cv2.EVENT_LBUTTONDOWN:
        click_point = (x, y)

cv2.namedWindow("Click to select object")
cv2.setMouseCallback("Click to select object", mouse_click)
while True:
    disp = first_frame.copy()
    if click_point is not None:
        cv2.circle(disp, click_point, 5, (0, 255, 255), -1)
    cv2.putText(disp, "Click object, then press ENTER",
                (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.imshow("Click to select object", disp)
    key = cv2.waitKey(1) & 0xFF
    if key in [13, 10]:  # Enter
        break
cv2.destroyWindow("Click to select object")

if click_point is None:
    print("No click point given")
    exit()

# --- find mask in first frame ---
img_rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
predictor.set_image(img_rgb)

point_coords = np.array([click_point])
point_labels = np.array([1])  # foreground

masks, scores, _ = predictor.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    multimask_output=True,
)

# pick the mask that contains the click
selected_mask = None
for m in masks:
    if m[click_point[1], click_point[0]]:  # y,x indexing
        selected_mask = m
        break

if selected_mask is None:
    print("No mask contains the clicked point")
    exit()

# get bounding box from mask
ys, xs = np.where(selected_mask)
x1, y1, x2, y2 = xs.min(), ys.min(), xs.max(), ys.max()
init_bbox = (x1, y1, x2 - x1, y2 - y1)  # (x, y, w, h)

# initialize tracker
if hasattr(cv2, "legacy"):
    tracker = cv2.legacy.TrackerCSRT_create()
else:
    tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, init_bbox)

# --- process video frame-by-frame ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # update tracker
    ok, box = tracker.update(frame)
    if not ok:
        cv2.putText(frame, "Tracking lost", (20, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.imshow("SAM Object Tracking", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        continue

    x, y, w, h = [int(v) for v in box]
    bbox_center = (x + w // 2, y + h // 2)

    # crop region for SAM
    crop_rgb = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2RGB)
    predictor.set_image(crop_rgb)

    # use a point prompt at the bbox center (relative to crop)
    rel_point = np.array([[w // 2, h // 2]])
    rel_label = np.array([1])

    masks, _, _ = predictor.predict(
        point_coords=rel_point,
        point_labels=rel_label,
        multimask_output=True,
    )

    # pick mask containing point
    obj_mask = None
    for m in masks:
        if m[h // 2, w // 2]:
            obj_mask = m
            break

    # draw mask in full-frame coordinates
    if obj_mask is not None:
        mask_full = np.zeros(frame.shape[:2], dtype=np.uint8)
        mask_full[y:y+h, x:x+w] = obj_mask.astype(np.uint8)
        contours, _ = cv2.findContours(mask_full, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, (0, 0, 255), 2)

    # draw tracking bbox
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.circle(frame, bbox_center, 4, (0, 255, 0), -1)

    cv2.imshow("SAM Object Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()